 The code in this notebook, **when run in order**, will:

#### (1) Produce a dataframe of tweets for a certain keyword (columns right now are, location and tweet text; can modify the search criteria as necessary (location lists, hashtags, etc.), saved as a csv

#### (2) Delete the duplicates/very similar spam tweets but leave one copy behind. 

#### (3) Clean the remaining tweets and saves as a csv

By the end of the notebook, you'll have a clean twitter dataframe that's ready for analysis 

In [1]:
import os
import tweepy
import pandas as pd
import time
import itertools
import numpy as np
from itertools import chain


In [2]:
consumer_key= '3emnsuDpqLBXlxD5UPUxVnxTt'
consumer_secret= 'ZPZBilbjbca7pYAmMv05hlNKsxLb4CJd6A5kNa2JNIs62BjF7m'
access_token= '1406213154-M77vYKXz7wIAUzzJkqLVzfDH0D6K11xMJkzGHzJ'
access_token_secret= 'iLcP4BuW4jWRYIHEiIZwsD5jgYlMY1aTmtSFwPQqbaQrX'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

## Specify tweet search/filtering criteria here:

In [179]:
search_words = "\"mask up\" OR #maskup OR maskup"
date_since = "2021-03-10"
new_search = search_words + " -filter:retweets"


## Set location labels and corresponding location coordinates and radii here:

In [180]:
## ** In the order of the counties listed in the google doc, and put in the Facebook chat

locs_labels = ['Austin TX',
'New York County NY',
 'County of San Francisco CA',
 'Hudson County NJ',
 'Suffolk County MA',
 'Philadelphia County PA',
 'Washington DC',
 'Alexandria VA',
 'Baltimore MD',
 'Cook County IL',
 'St. Louis County MO',
 'Milwaukee County WI',
 'Denver County CO',
 'Orange County CA',
 'Pinellas County FL',
 'Ramsey County MN',
 'Wayne County, MI',
 'Cuyahoga County OH',
 'Dallas County TX',
 'DeKalb County GA',
 'Salt Lake County UT',
 'Marion County IN',
 'Jefferson County KY',
 'Johnson County KS',
 'King County WA']


locs = ['29.88548725,-96.27736949,100mi',
    '40.7741618671,-73.9697971209,100mi',
    '37.7597150131,-122.693975573,100mi',
    '40.7309049975,-74.0759554422,100mi',
    '42.3486893925,-70.9856096697,100mi',
    '40.0076360466,-75.1339446794,100mi', 
    '38.9047737146,-77.0163026702,100mi',
    '38.8184535854,-77.0862060584,100mi',
    '39.443072852,-76.6163229502,100mi',
    '41.8954290221,-87.6461407931,100mi',
    '38.6406885662,-90.4433975603,100mi',
    '43.0154568883,-87.5805486607,100mi',
    '39.7621646287,-104.875801423,100mi',
    '33.6769134328,-117.776142822,100mi',
    '27.9026762041,-82.7395539464,100mi',
    '45.0170541432,-93.099606274,100mi',
    '42.2847454761,-83.2611328686,100mi',
    '41.6350590585,-81.7001959208,100mi',
    '32.7666300195,-96.7778770663,100mi',
    '33.7715468713,-84.226432416,100mi',
    '40.6673139262,-111.923601307,100mi',
    '39.7817096805,-86.1384716709,100mi',
    '38.1871910002,-85.6591577058,100mi',
    '38.8837685125,-94.8222610425,100mi',
    '47.4909231926,-121.83595173,100mi']

## Just zip locs_labels with locs and you'd have the coordinates mapped to their names

mapped_locations = list(zip(locs_labels, locs))

In [181]:
mapped_locations

[('Austin TX', '29.88548725,-96.27736949,100mi'),
 ('New York County NY', '40.7741618671,-73.9697971209,100mi'),
 ('County of San Francisco CA', '37.7597150131,-122.693975573,100mi'),
 ('Hudson County NJ', '40.7309049975,-74.0759554422,100mi'),
 ('Suffolk County MA', '42.3486893925,-70.9856096697,100mi'),
 ('Philadelphia County PA', '40.0076360466,-75.1339446794,100mi'),
 ('Washington DC', '38.9047737146,-77.0163026702,100mi'),
 ('Alexandria VA', '38.8184535854,-77.0862060584,100mi'),
 ('Baltimore MD', '39.443072852,-76.6163229502,100mi'),
 ('Cook County IL', '41.8954290221,-87.6461407931,100mi'),
 ('St. Louis County MO', '38.6406885662,-90.4433975603,100mi'),
 ('Milwaukee County WI', '43.0154568883,-87.5805486607,100mi'),
 ('Denver County CO', '39.7621646287,-104.875801423,100mi'),
 ('Orange County CA', '33.6769134328,-117.776142822,100mi'),
 ('Pinellas County FL', '27.9026762041,-82.7395539464,100mi'),
 ('Ramsey County MN', '45.0170541432,-93.099606274,100mi'),
 ('Wayne County, MI', 

In [182]:
len(locs) # ensure that locs and loc_labels lens match; should have through zip funct anyway

25

In [183]:
len(locs_labels)

25

In [319]:
'''
# Nassau county, NY
tweet_lst=[]
geoc="40.6546,73.5594,50mi"
txgeoc = '29.88548725,-96.27736949,100mi'
migeoc = '25.6112362,-80.55170587,50mi'
for tweet in tweepy.Cursor(api.search,q='#maskfree', lang="en", since=date_since, geocode=txgeoc, tweet_mode='extended').items(1000):
    print(tweet)
    if(tweet.coordinates != None):
        print('we got here')
        tweet_lst.append([tweet.id,tweet.coordinates[0],
                          tweet.coordinates[1],
                          tweet.user.screen_name,
                          tweet.user.name, tweet.text])
    else:
        print("it was none")
tweet_df = pd.DataFrame(tweet_lst, columns=['id', 'lat','long','username', 'name', 'tweet'])
'''

'\n# Nassau county, NY\ntweet_lst=[]\ngeoc="40.6546,73.5594,50mi"\ntxgeoc = \'29.88548725,-96.27736949,100mi\'\nmigeoc = \'25.6112362,-80.55170587,50mi\'\nfor tweet in tweepy.Cursor(api.search,q=\'#maskfree\', lang="en", since=date_since, geocode=txgeoc, tweet_mode=\'extended\').items(1000):\n    print(tweet)\n    if(tweet.coordinates != None):\n        print(\'we got here\')\n        tweet_lst.append([tweet.id,tweet.coordinates[0],\n                          tweet.coordinates[1],\n                          tweet.user.screen_name,\n                          tweet.user.name, tweet.text])\n    else:\n        print("it was none")\ntweet_df = pd.DataFrame(tweet_lst, columns=[\'id\', \'lat\',\'long\',\'username\', \'name\', \'tweet\'])\n'

In [320]:
#tweet_df

In [321]:
'''
### TEXAS TEST ####


tweet_lst=[]
geoc="40.6546,73.5594,50mi"
txgeoc = '29.88548725,-96.27736949,100mi'
migeoc = '25.6112362,-80.55170587,50mi'

    

# Creation of query method using parameters
try:
    tweets = tweepy.Cursor(api.search,q='mask free', lang="en", since=date_since, geocode=txgeoc, tweet_mode='extended').items(5) 
# Pulling information from tweets iterable object
    tweets_list = [[tweet.created_at, tweet.id, tweet.full_text] for tweet in tweets]
    tweets_sort = tweets_list.sort()
    no_dupes = list(k for k,_ in itertools.groupby(tweets_sort))
    print(tweets_list)
 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
    tweets_d = pd.DataFrame(no_dupes)

except BaseException as e:
    print('failed on_status,',str(e))

tweets_f = pd.DataFrame(no_dupes)
'''

'\n### TEXAS TEST ####\n\n\ntweet_lst=[]\ngeoc="40.6546,73.5594,50mi"\ntxgeoc = \'29.88548725,-96.27736949,100mi\'\nmigeoc = \'25.6112362,-80.55170587,50mi\'\n\n    \n\n# Creation of query method using parameters\ntry:\n    tweets = tweepy.Cursor(api.search,q=\'mask free\', lang="en", since=date_since, geocode=txgeoc, tweet_mode=\'extended\').items(5) \n# Pulling information from tweets iterable object\n    tweets_list = [[tweet.created_at, tweet.id, tweet.full_text] for tweet in tweets]\n    tweets_sort = tweets_list.sort()\n    no_dupes = list(k for k,_ in itertools.groupby(tweets_sort))\n    print(tweets_list)\n # Creation of dataframe from tweets list\n # Add or remove columns as you remove tweet information\n    tweets_d = pd.DataFrame(no_dupes)\n\nexcept BaseException as e:\n    print(\'failed on_status,\',str(e))\n\ntweets_f = pd.DataFrame(no_dupes)\n'

## The code that does the actual scraping by location:

In [184]:
# SCRAPING TWEETS FOR ALL LOCATIONS # #Kaylah added date info here

tweets_list = []
for loc in mapped_locations:
    try:
        tweets = tweepy.Cursor(api.search,q=new_search, lang="en", since=date_since, geocode=loc[1], tweet_mode='extended').items(1000) 
        tweets_list.append((loc[0], [[tweet.full_text, tweet.created_at.strftime("%d-%b-%Y")] for tweet in tweets]))
        print(tweets_list)
        tweets_df = pd.DataFrame(tweets_list)

    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)
    

[('Austin TX', [['Wearing a mask is still necessary to stop the spread of #COVID19. #MaskUp to protect yourself and loved ones. https://t.co/i31dw5cNMe', '19-Mar-2021'], ["Let's keep Edgewood safe for our students, staff and community. We can't wait to see all our kids back in class. We miss them! #COVID19  #MaskUp  #vaccination #safezone #EISDProud https://t.co/mGgH6QY4KG", '19-Mar-2021'], ['@EISDofSA Out of 1800 employees we’ve vaccinated over 1,066 people! Please keep encouraging others to keep Us safe! Show respect &amp; Mask up! Scholars are ready to return! We’re getting ready!\n@MarthaICastill1 @vote4ina @Menendez4Texas @raylopezfortx @Ron_Nirenberg @CommishJRod https://t.co/5Yoib5Dm8E', '19-Mar-2021'], ['Cogent op-ed in today’s San Antonio @ExpressNews by @UTAustin professor, Deborah Beck. Governor @GregAbbott_TX abdicated his responsibility and now 47,257 Texans are dead. Gone too soon. Shameful: https://t.co/LFSRLnT6GP #MaskUp #NoSeasTonto #VoteBlue2022 #AbbottFailedTexasAgai

[('Austin TX', [['Wearing a mask is still necessary to stop the spread of #COVID19. #MaskUp to protect yourself and loved ones. https://t.co/i31dw5cNMe', '19-Mar-2021'], ["Let's keep Edgewood safe for our students, staff and community. We can't wait to see all our kids back in class. We miss them! #COVID19  #MaskUp  #vaccination #safezone #EISDProud https://t.co/mGgH6QY4KG", '19-Mar-2021'], ['@EISDofSA Out of 1800 employees we’ve vaccinated over 1,066 people! Please keep encouraging others to keep Us safe! Show respect &amp; Mask up! Scholars are ready to return! We’re getting ready!\n@MarthaICastill1 @vote4ina @Menendez4Texas @raylopezfortx @Ron_Nirenberg @CommishJRod https://t.co/5Yoib5Dm8E', '19-Mar-2021'], ['Cogent op-ed in today’s San Antonio @ExpressNews by @UTAustin professor, Deborah Beck. Governor @GregAbbott_TX abdicated his responsibility and now 47,257 Texans are dead. Gone too soon. Shameful: https://t.co/LFSRLnT6GP #MaskUp #NoSeasTonto #VoteBlue2022 #AbbottFailedTexasAgai

[('Austin TX', [['Wearing a mask is still necessary to stop the spread of #COVID19. #MaskUp to protect yourself and loved ones. https://t.co/i31dw5cNMe', '19-Mar-2021'], ["Let's keep Edgewood safe for our students, staff and community. We can't wait to see all our kids back in class. We miss them! #COVID19  #MaskUp  #vaccination #safezone #EISDProud https://t.co/mGgH6QY4KG", '19-Mar-2021'], ['@EISDofSA Out of 1800 employees we’ve vaccinated over 1,066 people! Please keep encouraging others to keep Us safe! Show respect &amp; Mask up! Scholars are ready to return! We’re getting ready!\n@MarthaICastill1 @vote4ina @Menendez4Texas @raylopezfortx @Ron_Nirenberg @CommishJRod https://t.co/5Yoib5Dm8E', '19-Mar-2021'], ['Cogent op-ed in today’s San Antonio @ExpressNews by @UTAustin professor, Deborah Beck. Governor @GregAbbott_TX abdicated his responsibility and now 47,257 Texans are dead. Gone too soon. Shameful: https://t.co/LFSRLnT6GP #MaskUp #NoSeasTonto #VoteBlue2022 #AbbottFailedTexasAgai

[('Austin TX', [['Wearing a mask is still necessary to stop the spread of #COVID19. #MaskUp to protect yourself and loved ones. https://t.co/i31dw5cNMe', '19-Mar-2021'], ["Let's keep Edgewood safe for our students, staff and community. We can't wait to see all our kids back in class. We miss them! #COVID19  #MaskUp  #vaccination #safezone #EISDProud https://t.co/mGgH6QY4KG", '19-Mar-2021'], ['@EISDofSA Out of 1800 employees we’ve vaccinated over 1,066 people! Please keep encouraging others to keep Us safe! Show respect &amp; Mask up! Scholars are ready to return! We’re getting ready!\n@MarthaICastill1 @vote4ina @Menendez4Texas @raylopezfortx @Ron_Nirenberg @CommishJRod https://t.co/5Yoib5Dm8E', '19-Mar-2021'], ['Cogent op-ed in today’s San Antonio @ExpressNews by @UTAustin professor, Deborah Beck. Governor @GregAbbott_TX abdicated his responsibility and now 47,257 Texans are dead. Gone too soon. Shameful: https://t.co/LFSRLnT6GP #MaskUp #NoSeasTonto #VoteBlue2022 #AbbottFailedTexasAgai

[('Austin TX', [['Wearing a mask is still necessary to stop the spread of #COVID19. #MaskUp to protect yourself and loved ones. https://t.co/i31dw5cNMe', '19-Mar-2021'], ["Let's keep Edgewood safe for our students, staff and community. We can't wait to see all our kids back in class. We miss them! #COVID19  #MaskUp  #vaccination #safezone #EISDProud https://t.co/mGgH6QY4KG", '19-Mar-2021'], ['@EISDofSA Out of 1800 employees we’ve vaccinated over 1,066 people! Please keep encouraging others to keep Us safe! Show respect &amp; Mask up! Scholars are ready to return! We’re getting ready!\n@MarthaICastill1 @vote4ina @Menendez4Texas @raylopezfortx @Ron_Nirenberg @CommishJRod https://t.co/5Yoib5Dm8E', '19-Mar-2021'], ['Cogent op-ed in today’s San Antonio @ExpressNews by @UTAustin professor, Deborah Beck. Governor @GregAbbott_TX abdicated his responsibility and now 47,257 Texans are dead. Gone too soon. Shameful: https://t.co/LFSRLnT6GP #MaskUp #NoSeasTonto #VoteBlue2022 #AbbottFailedTexasAgai

failed on_status, 


KeyboardInterrupt: 

In [185]:
tweets_df

,0,1
0,Austin TX,[[Wearing a mask is still necessary to stop th...
1,New York County NY,[[MASK UP NYC! almost there! https://t.co/p8MM...
2,County of San Francisco CA,[[@Karakaradu @CMargaronis @_talia_t @doctor_e...
3,Hudson County NJ,[[MASK UP NYC! almost there! https://t.co/p8MM...
4,Suffolk County MA,[[@RandPaul claim that wearing a mask is theat...


In [186]:
tweets_df.to_csv('tweet7_dataset.csv') # saves the dataframe


################################################################################################

# Tweet df loading and cleaning

In [187]:
import os
import tweepy
import pandas as pd
import time
import itertools
import numpy as np
from itertools import chain
import regex as re
import unicodedata
import nltk

## Loads tweet dataframe, ensures datatypes are appropriate, flattens tweets into one list for ease, labels columns

In [188]:
tweets_df = pd.read_csv('tweet7_dataset.csv', converters={'1': eval}, index_col=0)
tweets_df['tweets_list'] = tweets_df['1'].apply(lambda x : np.array(x).flatten())
tweets_df.columns = ['county', 'original_tweet', 'tweets_list']

tweets_df.loc[0, 'county'] = 'Austin County TX'
tweets_df

,county,original_tweet,tweets_list
0,Austin County TX,[[Wearing a mask is still necessary to stop th...,[Wearing a mask is still necessary to stop the...
1,New York County NY,[[MASK UP NYC! almost there! https://t.co/p8MM...,[MASK UP NYC! almost there! https://t.co/p8MMh...
2,County of San Francisco CA,[[@Karakaradu @CMargaronis @_talia_t @doctor_e...,[@Karakaradu @CMargaronis @_talia_t @doctor_eo...
3,Hudson County NJ,[[MASK UP NYC! almost there! https://t.co/p8MM...,[MASK UP NYC! almost there! https://t.co/p8MMh...
4,Suffolk County MA,[[@RandPaul claim that wearing a mask is theat...,[@RandPaul claim that wearing a mask is theate...


In [327]:
# texas  --- 226 total tweets
#for index, tweet in enumerate(list(tweets_df['tweets_list'].iloc[0])):
#    print(index, tweet)
    

## Now, expands dataset so there is one row per tweet (before, all tweets for a county were in one row)


In [189]:
t_explode = tweets_df.explode('original_tweet')
t_explode

,county,original_tweet,tweets_list
0,Austin County TX,[Wearing a mask is still necessary to stop the...,[Wearing a mask is still necessary to stop the...
0,Austin County TX,"[Let's keep Edgewood safe for our students, st...",[Wearing a mask is still necessary to stop the...
0,Austin County TX,[@EISDofSA Out of 1800 employees we’ve vaccina...,[Wearing a mask is still necessary to stop the...
0,Austin County TX,[Cogent op-ed in today’s San Antonio @ExpressN...,[Wearing a mask is still necessary to stop the...
0,Austin County TX,[Well this is certainly one way to #maskup! ht...,[Wearing a mask is still necessary to stop the...
...,...,...,...
4,Suffolk County MA,[Our students took full advantage of the warm ...,[@RandPaul claim that wearing a mask is theate...
4,Suffolk County MA,"[City is revealing the ""mask up"" awards to you...",[@RandPaul claim that wearing a mask is theate...
4,Suffolk County MA,"[@jordanw_s Mask up buddy. Come on now!, 11-Ma...",[@RandPaul claim that wearing a mask is theate...
4,Suffolk County MA,[It’s a beautiful day! Mask up and get out! ...,[@RandPaul claim that wearing a mask is theate...


In [190]:
# Created new column for date

t_explode['date_tweeted'] = '0'

for tweet_ind, tweet_row, in t_explode.iterrows():
    tweet_row['date_tweeted'] = tweet_row['original_tweet'][1]

# cleaning date layout for easier matching to COVID-19 dataframe

for date_ind, date_row in t_explode.iterrows():
    date_alt = date_row['date_tweeted'].replace('-', ' ')
    date_alt = date_alt.split()
    date_alt[1] = '3'
    date_alt[2] = '21'
    date_alt[0], date_alt[1] = date_alt[1], date_alt[0]
    date_alt = "/".join(date_alt)
    date_row['date_tweeted'] = date_alt
t_explode.head()
    

,county,original_tweet,tweets_list,date_tweeted
0,Austin County TX,[Wearing a mask is still necessary to stop the...,[Wearing a mask is still necessary to stop the...,3/19/21
0,Austin County TX,"[Let's keep Edgewood safe for our students, st...",[Wearing a mask is still necessary to stop the...,3/19/21
0,Austin County TX,[@EISDofSA Out of 1800 employees we’ve vaccina...,[Wearing a mask is still necessary to stop the...,3/19/21
0,Austin County TX,[Cogent op-ed in today’s San Antonio @ExpressN...,[Wearing a mask is still necessary to stop the...,3/19/21
0,Austin County TX,[Well this is certainly one way to #maskup! ht...,[Wearing a mask is still necessary to stop the...,3/19/21


In [191]:
# tex = t_explode.loc[t_explode['county'] == 'Texas test']

# print(len(tex)) # confirm that this is 226; it is

In [192]:
t_explode['original_tweet'].iloc[56]

['Please stay this way San Antonio. I went to HEB earlier and only one idiot was without a mask. Please mask up and stay safe. Also Fuck Greg Abbott and Ted Cruz. https://t.co/Do9hjkgjhi',
 '17-Mar-2021']

***********
# See note:

I tried getting rid of duplicates a few ways; doing pairwise levenshtein distance comparisons between all the tweets took way too long, and creating a mask to then drop duplicate tweets was also complicated. I noticed that most very-similar-basically-duplicate tweets had the same text, but different handles and urls. So I took a slice of part of the text that was very likely to be shared between the similar tweets, created a new column in the dataset where each tweet-row got a corresponding tweet-slice, and then I deleted the rows where there were duplicate tweet-slices. As you can see, this resulted in a *lot* of data loss (3000+ tweets down to 850; see twitter_drop a few cells down), because many of them were duplicate/spam tweets.
***********

In [193]:
# converting to string data type 
t_explode["tweet_str"]= t_explode["original_tweet"].astype(str) 
  
# slicing till 2nd last element 
t_explode["tweet_slice"] = t_explode["tweet_str"].str.slice(50,1000) 
  
# display 
t_explode

,county,original_tweet,tweets_list,date_tweeted,tweet_str,tweet_slice
0,Austin County TX,[Wearing a mask is still necessary to stop the...,[Wearing a mask is still necessary to stop the...,3/19/21,['Wearing a mask is still necessary to stop th...,read of #COVID19. #MaskUp to protect yourself ...
0,Austin County TX,"[Let's keep Edgewood safe for our students, st...",[Wearing a mask is still necessary to stop the...,3/19/21,"[""Let's keep Edgewood safe for our students, s...",and community. We can't wait to see all our k...
0,Austin County TX,[@EISDofSA Out of 1800 employees we’ve vaccina...,[Wearing a mask is still necessary to stop the...,3/19/21,['@EISDofSA Out of 1800 employees we’ve vaccin...,"over 1,066 people! Please keep encouraging ot..."
0,Austin County TX,[Cogent op-ed in today’s San Antonio @ExpressN...,[Wearing a mask is still necessary to stop the...,3/19/21,['Cogent op-ed in today’s San Antonio @Express...,"by @UTAustin professor, Deborah Beck. Governo..."
0,Austin County TX,[Well this is certainly one way to #maskup! ht...,[Wearing a mask is still necessary to stop the...,3/19/21,['Well this is certainly one way to #maskup! h...,"://t.co/C9QhOAGGpO', '19-Mar-2021']"
...,...,...,...,...,...,...
4,Suffolk County MA,[Our students took full advantage of the warm ...,[@RandPaul claim that wearing a mask is theate...,3/11/21,['Our students took full advantage of the warm...,ther today! #roarpride #protectthepride #MaskU...
4,Suffolk County MA,"[City is revealing the ""mask up"" awards to you...",[@RandPaul claim that wearing a mask is theate...,3/11/21,"['City is revealing the ""mask up"" awards to yo...",artists who designed masks amid the pandemic.'...
4,Suffolk County MA,"[@jordanw_s Mask up buddy. Come on now!, 11-Ma...",[@RandPaul claim that wearing a mask is theate...,3/11/21,"['@jordanw_s Mask up buddy. Come on now!', '11...",-2021']
4,Suffolk County MA,[It’s a beautiful day! Mask up and get out! ...,[@RandPaul claim that wearing a mask is theate...,3/11/21,['It’s a beautiful day! Mask up and get out! ...,"rwoodSchools https://t.co/st0hcPjwGC', '11-Mar..."


### Can look through slice list to ensure that the slices look right

In [194]:
slice_list = t_explode['tweet_slice'].tolist()


for i, v in enumerate(slice_list):
    print(i,v)
#slice_list[184]

0 read of #COVID19. #MaskUp to protect yourself and loved ones. https://t.co/i31dw5cNMe', '19-Mar-2021']
1  and community. We can't wait to see all our kids back in class. We miss them! #COVID19  #MaskUp  #vaccination #safezone #EISDProud https://t.co/mGgH6QY4KG", '19-Mar-2021']
2  over 1,066 people! Please keep encouraging others to keep Us safe! Show respect &amp; Mask up! Scholars are ready to return! We’re getting ready!\n@MarthaICastill1 @vote4ina @Menendez4Texas @raylopezfortx @Ron_Nirenberg @CommishJRod https://t.co/5Yoib5Dm8E', '19-Mar-2021']
3  by @UTAustin professor, Deborah Beck. Governor @GregAbbott_TX abdicated his responsibility and now 47,257 Texans are dead. Gone too soon. Shameful: https://t.co/LFSRLnT6GP #MaskUp #NoSeasTonto #VoteBlue2022 #AbbottFailedTexasAgain https://t.co/wg10uQKH0Y', '19-Mar-2021']
4 ://t.co/C9QhOAGGpO', '19-Mar-2021']
5 ks for use against COVID-19 https://t.co/V6BhdCNLZo', '19-Mar-2021']
6  work are anything to go off, I’m just a well disguised z

1059 s! #Grateful #GetYourShot #MaskUp https://t.co/DbSk9xz84K', '12-Mar-2021']
1060 ure.\n\nFor $10, we can supply you with one of the best cloth masks we've ever used!\n\nNot only do they come in the ever-stylish black &amp; feature our snazzy logo, but every penny goes to #ChildrensMiracleNetwork!\n\n#covid19 #maskUP https://t.co/lZXNAPjAUQ", '12-Mar-2021']
1061 Joe Biden #MaskUp', '12-Mar-2021']
1062  way!  Mask up runners. https://t.co/JeaIjT2UB3', '12-Mar-2021']
1063  I wasn’t all by myself in my unemployment, my entire theatre community was there with me. Stay safe. We will get through this. We will all get to answer those annoying questions again soon. Mask up.', '12-Mar-2021']
1064 er" and be bored than these careless and restless, walkbout people I work with! #workplacecluster #WFH #COVIDIOT #COVID19 #MaskUp', '12-Mar-2021']
1065 still a #pandemic don’t let your guard down #COVID until everyone gets the #vaccine #GetVaccinated Stay safe Pareng Noli', '12-Mar-2021']
1066 ld Fa

1809 ', '14-Mar-2021']
1810 rt! and is simply a PART of the fight against Covid- 19. Mask up and social distance', '14-Mar-2021']
1811 wasn’t enforcing face mask rqmt with passenger in 2F. #MaskUp #disappointedUnited', '14-Mar-2021']
1812 hizer shot on Friday and I'm still gonna mask up and hold back on crowded places . Do you .", '14-Mar-2021']
1813 e continue to mask up. https://t.co/BXtdpXJwhb', '14-Mar-2021']
1814  2 masks made to fit to toddlers to adult with small size faces only)\n\nGet yours now! Only 2 left 😊\nSupport Small Businesses !!!\n\n#handmadewithlove\n#facemasksrequired\n#maskup https://t.co/ydOnyev8i4', '14-Mar-2021']
1815 igher so we can't see his evil eyes.  Playing obtuse as usual. https://t.co/jsEUOzrYou", '14-Mar-2021']
1816 den #MajorAndChamp #Champ #Major #DOTUS #WhiteHouseDogs #BarkingForBiden 🐾 @TheOvalPawffice @TheFirstDogs\n\n#WearAMask #SocialDistancing #WashYourHands #AvoidCrowds #COVID19💉#vaccine🥜#foodallergies #servicedog🐕\u200d🦺#dogsoftwitter https://

In [195]:
twitter_drop = t_explode.drop_duplicates(subset=['tweet_slice'], keep='last')
twitter_drop

,county,original_tweet,tweets_list,date_tweeted,tweet_str,tweet_slice
0,Austin County TX,[Wearing a mask is still necessary to stop the...,[Wearing a mask is still necessary to stop the...,3/19/21,['Wearing a mask is still necessary to stop th...,read of #COVID19. #MaskUp to protect yourself ...
0,Austin County TX,"[Let's keep Edgewood safe for our students, st...",[Wearing a mask is still necessary to stop the...,3/19/21,"[""Let's keep Edgewood safe for our students, s...",and community. We can't wait to see all our k...
0,Austin County TX,[@EISDofSA Out of 1800 employees we’ve vaccina...,[Wearing a mask is still necessary to stop the...,3/19/21,['@EISDofSA Out of 1800 employees we’ve vaccin...,"over 1,066 people! Please keep encouraging ot..."
0,Austin County TX,[Cogent op-ed in today’s San Antonio @ExpressN...,[Wearing a mask is still necessary to stop the...,3/19/21,['Cogent op-ed in today’s San Antonio @Express...,"by @UTAustin professor, Deborah Beck. Governo..."
0,Austin County TX,[Well this is certainly one way to #maskup! ht...,[Wearing a mask is still necessary to stop the...,3/19/21,['Well this is certainly one way to #maskup! h...,"://t.co/C9QhOAGGpO', '19-Mar-2021']"
...,...,...,...,...,...,...
4,Suffolk County MA,[Our students took full advantage of the warm ...,[@RandPaul claim that wearing a mask is theate...,3/11/21,['Our students took full advantage of the warm...,ther today! #roarpride #protectthepride #MaskU...
4,Suffolk County MA,"[City is revealing the ""mask up"" awards to you...",[@RandPaul claim that wearing a mask is theate...,3/11/21,"['City is revealing the ""mask up"" awards to yo...",artists who designed masks amid the pandemic.'...
4,Suffolk County MA,"[@jordanw_s Mask up buddy. Come on now!, 11-Ma...",[@RandPaul claim that wearing a mask is theate...,3/11/21,"['@jordanw_s Mask up buddy. Come on now!', '11...",-2021']
4,Suffolk County MA,[It’s a beautiful day! Mask up and get out! ...,[@RandPaul claim that wearing a mask is theate...,3/11/21,['It’s a beautiful day! Mask up and get out! ...,"rwoodSchools https://t.co/st0hcPjwGC', '11-Mar..."


In [196]:
twitter_drop['tweet_str'].iloc[89]


"['Here is why YOU STILL NEED TO Wear a MASK!\\n\\n #MASKUP\\n#MaskUpHou\\n\\nhttps://t.co/vdyMQLNFtH', '17-Mar-2021']"

In [197]:
twitter_drop['tweet_str'].iloc[88]


"['#htownrush @StephKHOU @BrandiKHOU @xmanwalton @chitakhou Good morning. Happy St.Patrick’s Day ☘️🍀🍀🍀Have corned beef and cabbage 🥬. Green beer 🍺. Little Rocky weather to start but nice ending. Stay positive. We will get through these hiccups. Mask up. Be kind.', '17-Mar-2021']"

## Cleans the tweets

In [198]:
df_clean = twitter_drop

In [199]:
# lower case
df_clean['tweet_text'] = df_clean['tweet_str'].str.lower()

# remove url links
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))

# remove url/website that didn't use http, is only checking for .com websites 
# so words that are seperated by a . are not removed
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
# remove @mention
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'@mention', '', x))
# remove {link}
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'{link}', '', x))
# remove &text; html chars
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'&[a-z]+;', '', x))
# [video]
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"\[video\]", '', x))
# remove all remaining characters that aren't letters, white space, or 
# the following #:)(/\='] that are used in emojis or hashtags
df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
df_clean

<ipython-input-199-89e914d04461>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['tweet_text'] = df_clean['tweet_str'].str.lower()
<ipython-input-199-89e914d04461>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['tweet_text'] = df_clean['tweet_text'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
<ipython-input-199-89e914d04461>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

,county,original_tweet,tweets_list,date_tweeted,tweet_str,tweet_slice,tweet_text
0,Austin County TX,[Wearing a mask is still necessary to stop the...,[Wearing a mask is still necessary to stop the...,3/19/21,['Wearing a mask is still necessary to stop th...,read of #COVID19. #MaskUp to protect yourself ...,'wearing a mask is still necessary to stop the...
0,Austin County TX,"[Let's keep Edgewood safe for our students, st...",[Wearing a mask is still necessary to stop the...,3/19/21,"[""Let's keep Edgewood safe for our students, s...",and community. We can't wait to see all our k...,let's keep edgewood safe for our students staf...
0,Austin County TX,[@EISDofSA Out of 1800 employees we’ve vaccina...,[Wearing a mask is still necessary to stop the...,3/19/21,['@EISDofSA Out of 1800 employees we’ve vaccin...,"over 1,066 people! Please keep encouraging ot...",'eisdofsa out of employees weve vaccinated ov...
0,Austin County TX,[Cogent op-ed in today’s San Antonio @ExpressN...,[Wearing a mask is still necessary to stop the...,3/19/21,['Cogent op-ed in today’s San Antonio @Express...,"by @UTAustin professor, Deborah Beck. Governo...",'cogent op-ed in todays san antonio expressnew...
0,Austin County TX,[Well this is certainly one way to #maskup! ht...,[Wearing a mask is still necessary to stop the...,3/19/21,['Well this is certainly one way to #maskup! h...,"://t.co/C9QhOAGGpO', '19-Mar-2021']",'well this is certainly one way to #maskup '-...
...,...,...,...,...,...,...,...
4,Suffolk County MA,[Our students took full advantage of the warm ...,[@RandPaul claim that wearing a mask is theate...,3/11/21,['Our students took full advantage of the warm...,ther today! #roarpride #protectthepride #MaskU...,'our students took full advantage of the warm ...
4,Suffolk County MA,"[City is revealing the ""mask up"" awards to you...",[@RandPaul claim that wearing a mask is theate...,3/11/21,"['City is revealing the ""mask up"" awards to yo...",artists who designed masks amid the pandemic.'...,'city is revealing the mask up awards to young...
4,Suffolk County MA,"[@jordanw_s Mask up buddy. Come on now!, 11-Ma...",[@RandPaul claim that wearing a mask is theate...,3/11/21,"['@jordanw_s Mask up buddy. Come on now!', '11...",-2021'],'jordanws mask up buddy come on now' '-mar-']
4,Suffolk County MA,[It’s a beautiful day! Mask up and get out! ...,[@RandPaul claim that wearing a mask is theate...,3/11/21,['It’s a beautiful day! Mask up and get out! ...,"rwoodSchools https://t.co/st0hcPjwGC', '11-Mar...",'its a beautiful day mask up and get out nor...


## Do a spot check to make sure that the data has been cleaned well:

In [200]:
df_clean['tweet_text'].iloc[5]

"'mask up how to choose and maintain the best masks for use against covid-  '-mar-']"

In [201]:
twitter_drop['tweet_str'].iloc[5]

"['Mask up! How to choose and maintain the best masks for use against COVID-19 https://t.co/V6BhdCNLZo', '19-Mar-2021']"

# Save as csv

In [ ]:
df_clean.to_csv('cleaned7_tweets.csv')

#########################################################################################################